<a href="https://colab.research.google.com/github/JoseGuarnizo/RAPIDS/blob/master/PANDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import os
import csv
import io
import json
import ast
import datetime as dt
from pandas.io.json import json_normalize 
from pandas import Series, DataFrame
from ast import literal_eval

In [ ]:
#Cargar data desde la maquina local a google colabority
from google.colab import files

carga = files.upload()

In [ ]:
#Proceso de Extracción

#Carga data en una variable
missing_values = [" "]
carga = pd.read_csv('df.csv', names='uno', na_values= missing_values)

In [ ]:
#convertir en list y despues en json
data2 = list(carga['u'])
da = []
for i in range(0,len(data2)):
    d = data2[i].split('.gz:')
    da.append(json.loads((d[1])))

In [ ]:
#Construcción DataFrame
new_data = pd.DataFrame(da)
new_data

In [ ]:
#Información de todo el DataFrame
new_data.columns
new_data.shape
new_data.dtypes

In [ ]:
#Verificación de tipo de datos por columna
new_data['event_type'].head()

In [ ]:
#Proceso de Transformación
#Eliminar columnas que no son necesarias
new_infor = new_data.drop(['accept_language', 'agent', 'page',
                           'session', 'event_type'], axis=1)

In [ ]:
#Verificar nuevamente la data
new_infor

In [ ]:
#Proceso Transformacion

#Dividir columna context

contextData = new_infor['context'].values.tolist()

new_contextData = pd.DataFrame(contextData, columns= ['course_user_tags',
                                                      'course_id', 'path',
                                                      'org_id', 'user_id'])

new_contextInfo = new_contextData.drop(['course_user_tags', 'course_id',
                                        'path'], axis=1)
new_contextInfo

In [ ]:
context_info = new_contextInfo.drop(["user_id"], axis=1)
context_info

In [ ]:
#ver datos columna event
new_infor['event']

In [ ]:
#Proceso de Transformación

#Dividir columna event

dat = new_infor['event'].values.tolist()
valore = []

for i in range(0,len(dat)):
    if (type(dat[i])==str):
        try:
            valor = json.loads(dat[i])
        except ValueError as e:
            print('mal')
        dat[i]=valor
        valore.append(dat[i])
    else:
        valore.append(dat[i])
valore

In [ ]:
#Creación nuevo dataframe de subcolumnas de la columna Event
nuevoEvent = pd.DataFrame(valore)
nuevoEvent

In [ ]:
#Eliminar columnas innecesarias de data event
new_eventInfo = nuevoEvent.drop(['course_id', 'mode'], axis=1)
new_eventInfo.head()

In [ ]:

#Eliminar columna GET no tiene datos importantes de data event
ver = new_eventInfo.drop(['GET'], axis=1)
ver

In [ ]:
#Dividiendo columna POST e incorporando a data event general
nuevovalorEventPost = pd.concat([ver.drop(['POST'], axis=1),
           ver.POST.apply(pd.Series)], axis=1)

newDataEvent = nuevovalorEventPost.drop([0], axis=1)
newDataEvent

In [ ]:
new_infor.equals(context_info)

False

In [ ]:
#Incorporar columna nueva Event a la data general
frames = [new_infor, newDataEvent, context_info]

generalData = pd.concat(frames,  axis=1, sort=False)
generalData.head()

In [ ]:
#Data trata
generalDataNew = generalData.drop(['context', 'event',
                                   'honor_code', 'password'], axis=1)
generalDataNew.head()

In [ ]:
generalDataNew.columns
generalDataNew.shape
generalDataNew.isna().sum()
generalDataNew.dtypes

In [ ]:
#calcular la media columna user_id
median = generalDataNew['user_id'].median()
generalDataNew['user_id'].fillna(median, inplace=True)


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:
generalDataNew

,username,event_source,name,time,host,referer,ip,user_id,analytics,city,country,course_id,email,enrollment_action,gender,goals,level_of_education,mailing_address,name,notifyOnError,remember,username,year_of_birth,org_id
0,,server,edx.course.enrollment.activated,2017-11-10T14:34:24.674569+00:00,,,,258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTPL
1,,server,NaN,2017-11-10T15:01:30.054897+00:00,opencampus.utpl.edu.ec,,66.249.64.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTPL
2,,server,NaN,2017-11-10T15:32:28.277486+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/,190.110.206.116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTPL
3,,server,NaN,2017-11-10T15:32:29.041853+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,190.110.206.116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,,server,NaN,2017-11-10T16:58:38.328698+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/,172.17.24.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,dicc,server,NaN,2017-11-21T22:09:11.659918+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.125.216.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
996,dicc,server,NaN,2017-11-21T22:10:03.295806+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.125.216.91,NaN,NaN,NaN,NaN,[course-v1:UTPL+EFHE-Ed7+2017_DIC],NaN,[enroll],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
997,dicc,server,edx.course.enrollment.activated,2017-11-21T22:10:03.370828+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.125.216.91,13907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTPL
998,dicc,server,NaN,2017-11-21T22:10:03.460488+00:00,opencampus.utpl.edu.ec,http://opencampus.utpl.edu.ec/courses/course-v...,200.125.216.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
